## Mining Big Datasets - Data Mining Assignment 1

### Assignment 1: User Similarity using Jaccard Distance, Min Hashing, and LSH

In this assignment, we will explore the use of Jaccard distance, min hashing, and Locality Sensitive Hashing (LSH) in the context of user similarity in a movie rating dataset. The dataset used for this assignment contains 100,000 ratings from 943 users on 1,682 movies. We will perform various tasks to compute user similarity and evaluate the effectiveness of different techniques. The tasks include importing and pre-processing the dataset, computing the exact Jaccard similarity of users, computing similarity using Min-hash signatures, and locating similar users using LSH index.

While the current analysis will be done on [Jupyter Notebook](http://jupyter.org/) and in [Python 3.10.0](https://www.python.org/downloads/release/python-3100/).

---

####  Team members:

---
> Dimitrios Matsanganis <br />
> Academic ID: f2822212 <br />
> MSc Business Analytics 2022-2023 FT <br />
> Athens University of Economics and Business <br />
> dmatsanganis@gmail.com, dim.matsanganis@aueb.gr

---

> Foteini Nefeli Nouskali <br />
> Academic ID: f2822213 <br />
> MSc Business Analytics 2022-2023 FT <br />
> Athens University of Economics and Business <br />
> fn.nouskali@gmail.com, fot.nouskali@aueb.gr

---

### Task 1: Import and Pre-process the Dataset with Users
---

We will start by downloading the movieLens dataset, which includes information about users, movies, and ratings. The dataset consists of three files: 
- users.txt 
- movies.txt
- ratings.txt

The users.txt file contains user-related information such as user ID, age, gender, occupation, and postcode. The movies.txt file contains movie-related information including movie ID, title, and other details. The ratings.txt file contains user IDs, movie IDs, ratings (on a scale of 1-5), and timestamps.

For this assignment, we will focus on the set of movies that each user has rated and not the actual ratings. We will preprocess the dataset by extracting the movie IDs for each user and organizing the data in a suitable format for further analysis. Any necessary conversions or processing steps will be described in detail, along with the reasons behind them.

As the initial step, we import the [Pandas](https://pandas.pydata.org/) library, which is a powerful data manipulation and analysis library in Python. We will use it to read and process the dataset. Therefore, the first step is to import the library.

In [1]:
import pandas as pd

The second step is to read the dataset files using pd.read_csv function from pandas. We provide the file names and the appropriate separator (`|` for users.txt and movies.txt, and `\t` for ratings.txt) and specify column names for each dataframe. 

The names parameter is used to assign column names to the dataframes. We store the information from each file in separate dataframes (users_df, movies_df, and ratings_df).

In [2]:
# Read users.txt and store it as users_df.
users_df = pd.read_csv('users.txt', sep='|', header=None, names=['userid', 'age', 'gender', 'occupation', 'postcode'])

# Preview a sample of the dataframe.
users_df.sample(10)

,userid,age,gender,occupation,postcode
924,925,18,F,salesman,49036
255,256,35,F,none,39042
345,346,34,M,other,76059
359,360,51,M,other,98027
508,509,23,M,administrator,10011
399,400,33,F,administrator,78213
847,848,46,M,engineer,02146
299,300,26,F,programmer,55106
435,436,30,F,administrator,17345
158,159,23,F,student,55346


In [3]:
# Read movies.txt and store it as movies_df.
movies_df = pd.read_csv('movies.txt', sep='|', header=None, encoding='latin-1', usecols=[0, 1])

# Rename the columns to 'movieid' and 'movie_title'.
movies_df.columns = ['movieid', 'movie_title']

# Preview a sample of the dataframe.
movies_df.sample(10)

,movieid,movie_title
361,362,Blues Brothers 2000 (1998)
755,756,Father of the Bride Part II (1995)
1299,1300,'Til There Was You (1997)
335,336,Playing God (1997)
173,174,Raiders of the Lost Ark (1981)
501,502,Bananas (1971)
132,133,Gone with the Wind (1939)
1182,1183,"Cowboy Way, The (1994)"
1207,1208,Kiss of Death (1995)
1278,1279,Wild America (1997)


In [4]:
# Read ratings.txt and store it as ratings_df.
ratings_df = pd.read_csv('ratings.txt', sep='\t', header=None, names=['userid', 'movieid', 'rating', 'timestamp'])

# Preview a sample of the dataframe.
ratings_df.sample(10)

,userid,movieid,rating,timestamp
48237,655,740,3,888474713
81873,425,11,3,878737981
31818,417,855,2,879647450
91511,497,1041,3,879310473
16020,303,384,3,879485165
91541,149,333,1,883512591
54994,675,937,1,889490151
70121,712,486,4,874730521
34552,254,21,3,886474518
37050,378,365,2,880318158


Reading the `movies_df` we encountered an issue regarding non-standard characters in the file. To handle this, we used the encoding parameter (`latin-1`), which is used because it supports a wider range of characters compared to the default UTF-8 encoding. This ensures that any non-standard characters in the file are handled correctly during the reading process. 

Then, the `usecols=[0, 1]` parameter is used to select only the first two columns from the file. This is done to focus on the columns that are important for the analysis, discarding any additional columns that are not required - following the assignmen's instructions. After reading the file, the column names are updated to `movieid` and `movie_title`.

For the other two datasets `users.txt` and `ratings.txt`, no encoding issues were encountered because they didn't contain characters incompatible with the default UTF-8 encoding. Hence, the `latin-1` encoding was not required for those datasets. 

Finally, all three datasets `users.txt`, `movies.txt`, and `ratings.txt` have been successfully loaded and previewed according to the provided instructions.

Then, we create the user_movie_df dataframe by selecting only the relevant columns (userid and movieid) from ratings_df and dropping any duplicate rows.

In [5]:
# Select the relevant columns from ratings_df (userid and movieid) and drop duplicates.
ratings_df = ratings_df[['userid', 'movieid']]

# Preview a sample of the dataframe.
ratings_df.sample(10)

,userid,movieid
20201,250,50
90010,734,164
17542,374,552
17360,195,271
49851,655,320
27550,390,319
87058,815,1204
75492,134,678
29275,358,127
77788,271,178


Now we will merge the three dataframes `users_df`, `movies_df`, and `ratings_df` into a single dataframe by combining the relevant columns from each dataframe. The common column between these dataframes is `userid` from users_df and `movieid` from movies_df and ratings_df. The resulting dataframe will be named `merged_df`.

To be more precise, first, we merge the ratings_df dataframe with the users_df dataframe. We select specific columns (userid, gender, age, occupation, and postcode) from the users_df dataframe to include in the merged dataframe. The merging is performed based on the common column 'userid'.

Next, we merge the resulting dataframe with the movies_df dataframe. We select the columns 'movieid' and 'movie_title' from the movies_df dataframe to include in the merged dataframe. The merging is again performed based on the common column 'movieid'.

The merged dataframe merged_df now contains a combination of information from all three original dataframes. It includes columns such as 'userid', 'gender', 'age', 'occupation', 'postcode', 'movieid', and 'movie_title'.

To ensure the correctness of the merging process, we preview a sample of 10 randomly selected rows from the merged_df dataframe. This provides a glimpse of the merged data and allows us to verify that the merging operation was successful.

By merging the dataframes, we create a consolidated dataset that brings together user information, movie details, and corresponding ratings. This combined dataset can be used for further analysis and insights.

In [6]:
# Merge the dataframes.
merged_df = pd.merge(ratings_df, users_df[['userid', 'gender', 'age', 'occupation', 'postcode']], on='userid')
merged_df = pd.merge(merged_df, movies_df[['movieid', 'movie_title']], on='movieid')

# Preview a sample of the merged dataframe.
merged_df.sample(10)

,userid,movieid,gender,age,occupation,postcode,movie_title
16771,190,291,M,30,administrator,95938,Absolute Power (1997)
49381,279,763,M,33,programmer,85251,Happy Gilmore (1996)
27051,246,451,M,19,student,28734,Grease (1978)
48855,233,197,M,38,engineer,98682,"Graduate, The (1967)"
36104,587,310,M,26,other,14216,"Rainmaker, The (1997)"
92223,113,292,M,47,executive,95032,Rosewood (1997)
42070,738,69,M,35,technician,95403,Forrest Gump (1994)
36719,13,7,M,47,educator,29206,Twelve Monkeys (1995)
86363,894,1381,M,47,educator,74075,Losing Chase (1996)
74192,305,60,M,23,programmer,94086,Three Colors: Blue (1993)


### Task 2: Compute Exact Jaccard Similarity of Users
---

To assess the similarity between users, we will compute the exact Jaccard Similarity for all pairs of users. The Jaccard Similarity measures the similarity between two sets by calculating the size of their intersection divided by the size of their union. We will compute the Jaccard Similarity for all unique pairs of users and output only those pairs with a similarity score of at least 0.5 (50% or higher). Additionally, we will identify the movie titles that the most similar pair of users have both seen.

To be more preceise, the first step is to create a set of movies for each user. We can do so if we iterate through the merged dataframe from Task 1 (merged_df) and for each unique user, create a set of movies they have seen.

In [7]:
# Create a set of movies for each user.
user_movies = {}

# A for-loop statement to implement the iteratively procedure.
for row in merged_df.itertuples():
    userid = getattr(row, 'userid')
    movieid = getattr(row, 'movieid')
    
    if userid not in user_movies:
        user_movies[userid] = set()
    user_movies[userid].add(movieid)

The next step is to compute Jaccard similarity for each pair of users. Again we will iterate through all possible pairs of users and calculate their Jaccard similarity using the formula: 

`Jaccard Similarity = Intersection / Union`, 

where `Intersection` is the number of movies both users have seen and `Union` is the total number of unique movies seen by both users.

Finally, we will store the pairs of users with a similarity score of at least 0.5. Thus, through an if-condition statement we will identify the pairs of users that have a Jaccard similarity of at least 0.5 (50%) and store their userids and the corresponding similarity score. Meanwhile, we take care of the duplicate reversed pairs.

In [8]:
# Through this inner for-loop statement we aim to compute Jaccard similarity for each pair of users.
similar_pairs = []
duplicates = set()

for user1 in user_movies:
    
    for user2 in user_movies:
        
        if user1 != user2 and (user2, user1) not in duplicates:
            movies1 = user_movies[user1]
            movies2 = user_movies[user2]
            
            intersection = len(movies1.intersection(movies2))
            union = len(movies1.union(movies2))
            
            jaccard_similarity = intersection / union
            
            # An If-condition to store the pairs with similarity >= 0.5.
            if jaccard_similarity >= 0.5:
                similar_pairs.append((user1, user2, jaccard_similarity))
                duplicates.add((user1, user2))
                
# Print the total number of similar pairs with Jaccardi above 50% existing in the list. 
print("The number of users' pairs with Jaccard similarity index above 0.5 is", len(similar_pairs), "and are presented below:")
print()

# Sort the pairs based on the Jaccard score.
similar_pairs = sorted(similar_pairs, key=lambda x: x[2], reverse = True)

# Preview the results.
for i in similar_pairs:
    print("User {} and User {}: {}".format(i[0], i[1], i[2]))

The number of users' pairs with Jaccard similarity index above 0.5 is 10 and are presented below:

User 408 and User 898: 0.8387096774193549
User 328 and User 788: 0.6729559748427673
User 489 and User 587: 0.6299212598425197
User 600 and User 826: 0.5454545454545454
User 451 and User 489: 0.5333333333333333
User 674 and User 879: 0.5217391304347826
User 554 and User 764: 0.5170068027210885
User 197 and User 826: 0.512987012987013
User 197 and User 600: 0.5
User 879 and User 800: 0.5


As we can see from the above output, there are 10 pairs of user with Jaccard similarity over 0.5 value.

The final step to conclude this task is to identify the pair of users with the highest similarity score and output the results properly. To do so we identified the pair of users with the highest Jaccard similarity score from the `similar_pairs` list. The `max()` function is used with a lambda function as the key to compare the similarity scores and select the maximum value. The most similar pair is stored in `most_similar_pair`, and the user IDs and similarity score are assigned to `user1_id`, `user2_id`, and `similarity_score`, respectively.

Then, to find the movie titles that the most similar pair of users has seen, we retrieved the set of movies seen by each user using their respective IDs - `user1_id` and `user2_id` - from the `user_movies` dictionary. The `intersection()` method is used to find the **common movies between the two sets**, and the result is stored in `common_movies`. Finally, we printed the information of the most similar pair of users and the movie titles they have seen in common, in a readable format. 

In [9]:
# Find the most similar pair of users.
most_similar_pair = max(similar_pairs, key=lambda x: x[2])
user1_id, user2_id, similarity_score = most_similar_pair

# Find the movie titles that the most similar pair of users has seen.
user1_movies = user_movies[user1_id]
user2_movies = user_movies[user2_id]
common_movies = user1_movies.intersection(user2_movies)

# Output the results.
print("Most Similar Pair of Users:\n")
print("User 1 ID:", user1_id)
print("User 2 ID:", user2_id)
print("Jaccard Similarity Score:", similarity_score)
print("\nCommon Movie Titles:")
for movie_id in common_movies:
    movie_title = movies_df[movies_df['movieid'] == movie_id]['movie_title'].values[0]
    print("  -", movie_title)

Most Similar Pair of Users:

User 1 ID: 408
User 2 ID: 898
Jaccard Similarity Score: 0.8387096774193549

Common Movie Titles:
  - Contact (1997)
  - Gattaca (1997)
  - Starship Troopers (1997)
  - Good Will Hunting (1997)
  - Indian Summer (1996)
  - Mouse Hunt (1997)
  - English Patient, The (1996)
  - Scream (1996)
  - Rocket Man (1997)
  - Air Force One (1997)
  - L.A. Confidential (1997)
  - Jackal, The (1997)
  - Rainmaker, The (1997)
  - Midnight in the Garden of Good and Evil (1997)
  - Titanic (1997)
  - Apt Pupil (1998)
  - Everyone Says I Love You (1996)
  - Lost Highway (1997)
  - Cop Land (1997)
  - Conspiracy Theory (1997)
  - U Turn (1997)
  - Wag the Dog (1997)
  - Spawn (1997)
  - Saint, The (1997)
  - Tomorrow Never Dies (1997)
  - Kolya (1996)


The results of the analysis reveal the most similar pair of users based on the Jaccard similarity score and the movie titles they have seen in common. The most similar pair of users consists of **User 1 with ID 408 and User 2 with ID 898**. Their Jaccard similarity score is calculated to be approximately **0.8387, indicating a high degree of similarity in their movie preferences, between these 2 users**.

These two users share a substantial number of movies in common, suggesting similar tastes in movies. The common movie titles include "Contact" (1997), "Gattaca" (1997), "Starship Troopers" (1997), "Good Will Hunting" (1997), "Titanic" (1997), and many others. This indicates that User 1 (ID 408) and User 2 (ID 898) have likely watched and enjoyed these movies, indicating potential similar preferences and interests in the movie domain.

### Task 3: Compute Similarity using Min-Hash Signatures
---

In this step, we will compute Min-Hash signatures for each user and use them to evaluate user similarity. Min-Hashing is a technique that approximates Jaccard Similarity by creating a set of hash functions and selecting the minimum hash value for each user. We will use a family of hash functions, ha,b(x) = (ax + b) mod R, where a and b are random integers and R is a large prime number.

We will evaluate the Min-Hashing technique using **50**, **100**, and **200 hash functions**. For each value, we will output the pairs of users with an estimated similarity of at least **0.5**. Additionally, we will calculate the number of false positives and false negatives against the exact Jaccard Similarity. We will run the experiment five times using different hash functions and report the **average false positives and false negatives**. We will also analyze how the number of hash functions affects the false positive and false negative outputs.

Codewise, we first import the necessary libraries, including `random` for generating random numbers and `numpy` for numerical operations.

In [10]:
import random
import numpy as np

Then, we set a random seed to ensure reproducibility of the results. This means that each time the code is run, it will produce the same random numbers. 

Furthermore, we define a list of `num_hash_functions` containing the numbers **50**, **100**, and **200**, representing the different numbers of hash functions we want to evaluate. Additionally, we define a large prime number R to be used in the hash functions.

In [11]:
# Set the random seed for reproducibility of the same results.
random.seed(50795)

# Define the number of hash functions.
num_hash_functions = [50, 100, 200]

# Define the large prime number.
R = 10000001

Afterwards, we define a function `generate_hash_functions` that takes the number of hash functions `num_functions` as input and generates random values for `a` and `b` in the hash function *ha,b(x) = (ax + b) mod R*. The function returns a list of hash functions.

In [12]:
# Function to generate hash functions.
def generate_hash_functions(num_functions):
    hash_functions = []
    for _ in range(num_functions):
        a = random.randint(1, R)
        b = random.randint(0, R)
        hash_functions.append((a, b))
    return hash_functions

In the following cell command, we create a dictionary `user_movies` to store the movies for each user and we define a function `compute_minhash_signature` that calculates the **Min-hash signature for a user**. It takes the `user_movies` (set of movie IDs) and `hash_functions` as input. 

The function initializes a signature list with infinity values and iterates over each `movie ID` for the user. For each hash function, it computes the hash value and updates the signature if the new hash value is *smaller*. The function returns the final `Min-hash signature` as *signature*.

In [13]:
# Create a dictionary to store movies for each user.
user_movies = {}

# Function to compute Min-hash signature for a user.
def compute_minhash_signature(user_movies, hash_functions):
    signature = [float('inf')] * len(hash_functions)
    for movie_id in user_movies:
        for i, (a, b) in enumerate(hash_functions):
            hash_value = (a * movie_id + b) % R
            if hash_value < signature[i]:
                signature[i] = hash_value
    return signature

We also create the `calculate_estimated_similarity` function, which is used within the evaluation process to compare the Min-hash signatures of different users and determine their estimated similarity. This function plays a crucial role in computing the similarity scores used to identify pairs of users with a similarity of at least 0.5.

In [14]:
# Function to calculate estimated similarity between users.
def calculate_estimated_similarity(user1_signature, user2_signature):
    num_matching = sum(user1_signature[i] == user2_signature[i] for i in range(len(user1_signature)))
    return num_matching / len(user1_signature)

As the next step, we iterate over the rows of the merged dataframe of the Task 1 (merged_df) and populate the dictionary by adding movie IDs to the corresponding user's set of movies.

In [15]:
# Iterate over the merged_df rows to populate user_movies dictionary.
for row in merged_df.itertuples():
    userid = getattr(row, 'userid')
    movieid = getattr(row, 'movieid')

    if userid not in user_movies:
        user_movies[userid] = set()
    user_movies[userid].add(movieid)

Then, we define the function `evaluate_similarity` that takes the number of hash functions `num_functions` as input. Within the function, we initialize variables for false positives and false negatives. The evaluation is run five times for averaging.

Inside each evaluation run, we generate hash functions that was previously implemented using `generate_hash_functions`. We also create a dictionary `user_signatures` to store the *Min-hash signatures* for each user. We compute the Min-hash signatures using `compute_minhash_signature` for each user's set of movies.

We then compare the Min-hash signatures and evaluate the similarity between users. We calculate the estimated similarity using `calculate_estimated_similarity` and the `exact Jaccard similarity`. Based on the comparison, we update the false positives and false negatives counters.

After all evaluations, finally we calculate the average **false positives** and **false negatives** and return them.

In [16]:
# Function to evaluate false positives and false negatives.
def evaluate_similarity(num_functions):
    false_positives = 0
    false_negatives = 0
    # true_positives = 0
    # true_negatives = 0

    # Run the evaluation 5 times for averaging.
    for _ in range(5):  

        # Generate hash functions.
        hash_functions = generate_hash_functions(num_functions)

        # Create dictionaries to store Min-hash signatures.
        user_signatures = {}

        # Compute Min-hash signatures for each user.
        for user_id, movies in user_movies.items():
            signature = compute_minhash_signature(movies, hash_functions)
            user_signatures[user_id] = signature

        # Compare Min-hash signatures and evaluate similarity.
        for user1_id, signature1 in user_signatures.items():
            for user2_id, signature2 in user_signatures.items():
                if user1_id != user2_id:
                    estimated_similarity = calculate_estimated_similarity(signature1, signature2)
                    exact_similarity = len(user_movies[user1_id].intersection(user_movies[user2_id])) / len(user_movies[user1_id].union(user_movies[user2_id]))
                    if estimated_similarity >= 0.5 and exact_similarity < 0.5:
                        false_positives += 1
                    elif estimated_similarity < 0.5 and exact_similarity >= 0.5:
                        false_negatives += 1
                        
                    # For evaluation purposes.
                    #  elif estimated_similarity >= 0.5 and exact_similarity >= 0.5:
                    #      true_positives += 1
                    #  elif estimated_similarity < 0.5 and exact_similarity < 0.5:
                    #      true_negatives += 1

    # Divide by 5 for the average of the 5 runs and by 2 since we want to have a 
    # picture regarding the single and not duplicate comparisons of users.
    avg_false_positives = false_positives / (2 * 5)
    avg_false_negatives = false_negatives / (2 * 5) 
    
    # For evaluation purposes.   
    # true_positives = true_positives / (2 * 5)
    # true_negatives = true_negatives / (2 * 5)
    # return avg_false_positives, avg_false_negatives, true_positives, true_negatives

    return avg_false_positives, avg_false_negatives

Then, we perform the evaluation of Min-hashing for each number of hash functions in `num_hash_functions (50, 100, and 200)`. 

After that we call the `evaluate_similarity` function for each number of hash functions and retrieve the **average false positives and false negatives**.

We then print the results in a readble format.

In [17]:
# Perform the evaluation for each number of hash functions.
for num_functions in num_hash_functions:
    
    # For evaluation purposes.   
    # avg_false_positives, avg_false_negatives, true_positives, true_negatives = evaluate_similarity(num_functions)
    
    # Call the function implemented to find the pairs similatity.
    avg_false_positives, avg_false_negatives = evaluate_similarity(num_functions)

    print("Number of Hash Functions:", num_functions)
    print("Average False Positives:", avg_false_positives)
    print("Average False Negatives:", avg_false_negatives)

    # For evaluation purposes.   
    # print("Average True Positives:", true_positives)
    # print("Average True Negatives:", true_negatives)

    print()

Number of Hash Functions: 50
Average False Positives: 204.4
Average False Negatives: 1.8

Number of Hash Functions: 100
Average False Positives: 36.4
Average False Negatives: 2.0

Number of Hash Functions: 200
Average False Positives: 10.0
Average False Negatives: 2.2



The results of the evaluation of Min-hash signatures with different numbers of hash functions are the following:

*For 50 hash functions:*

- Average False Positives: 204.4
- Average False Negatives: 1.8

*For 100 hash functions:*

- Average False Positives: 36.4
- Average False Negatives: 2.0

*For 200 hash functions:*

- Average False Positives: 10.0
- Average False Negatives: 2.2

The false positives represent the cases where the estimated similarity between users based on the Min-hash signatures is at least 0.5, but the exact Jaccard similarity is less than 0.5.

On the other hand, false negatives represent the cases where the estimated similarity is less than 0.5, but the exact Jaccard similarity is at least 0.5.

**Conclusion**

For **50** hash functions the average false positives are relatively **high at 204.4**, indicating that there is a significant number of pairs of users that have a high estimated similarity but a low exact Jaccard similarity. This suggests that the Min-hash signatures with 50 hash functions **may not be very accurate in capturing the true similarity between users**.

The average false negatives are significantly low at **1.8**, indicating that the Min-hash signatures are generally effective in capturing the cases where users have a low estimated similarity but a high exact Jaccard similarity. The false negatives suggest that the Min-hash signatures perform well in identifying dissimilar users.

When we elevate the number of hash functions to **100**, the average false positives **decrease significantly to 36.4 compared to 50 hash functions**. This indicates an improvement in accurately identifying pairs of users with high estimated similarity and low exact Jaccard similarity.
The average false negatives **slightly rises to 2.0**, indicating that this non-substantial difference does not imply that the increase in the min hash signatures length deteriorates the non-idetifiable user pairs and consequently the amelioration in the the false pocitive incidents is a far more important improvement.

Finally, when we set the number of hash functions to **200**, the average false positives further **decrease to 10.0, suggesting a higher accuracy in identifying pairs of users with high estimated similarity but low exact Jaccard similarity compared to 100 hash functions**.
The average false negatives **slightly rises again to 2.2**, indicating that the Min-hash signatures continue to perform well in capturing cases of low estimated similarity but high exact Jaccard similarity.

Overall, **increasing the number of hash functions from 50 to 200 leads to a reduction in false positives and to a non-substantial increase to false negatives that should not be taken into consideration as it is below one pair increase on average, resulting in more accurate similarity estimation overall using Min-hash signatures**. A larger number of hash functions provides finer-grained hashing and improves the precision of similarity estimation. However, it's important to *strike a balance as increasing the number of hash functions also increases the computational cost*. Therefore, choosing an appropriate number of hash functions requires considering the *trade-off between accuracy and computational efficiency*. 

Therefore, depending on the computation cost the 100 hash functions choice is probably the best one. However, we do not have any particular information and timewise demand so, the comparison lead to the same results, therefore we cannot select the optimum between the two, so we assume that the best one is the more efficient one - at least resultwise - so it is the solution containing the 200 hash functions.

### Task 4: Locate Similar Users using LSH Index
---

In this task, we will implement Locality Sensitive Hashing (LSH) using a set of 200 hash functions. We will break the Min-Hash signatures into bands with a specific number of rows and specifically number of min hash values per band. Two instances of LSH will be used, namely:
- LSH instance 1 with b = 25 and r = 8
- LSH instance 2 with b = 40 and r = 5

Using each LSH instance, we will locate pairs of users that share at least one identical band. These pairs present a high probabity of presenting similarity in a great extent. In the next step the range of the aforementioned user pairs are the only possible combinations that the Jaccard Similarity will be calculated so as to actually identify if they share similarity at least or above 50%. The LSH recommended similar pairs of users will be tested as regard as they exact Jaccards Similarity Index and the pairs with exact index value above 50% will be retained as true positives pairs that LSH method conducted. We will report the number of true pairs returned (true positives) and the number of similarity evaluations performed that correspond to the number of similar pairs the LSH method proposed as similar. The experiment will be repeated five times with different hash functions, and the averages of the true positives and similarity evaluations will be reported.

Finally, based on the reported results, we will discuss the advantages and disadvantages of using LSH compared to directly comparing users based on their true representations sets.

To be more precise, codewise we start by importing libraries and initialize components. 

In [18]:
import random
import numpy as np

# Set the random seed for reproducibility purposes.
random.seed(129537555)

# Set R to be a large prime number.
R = 1000001

# Set the similarity threshold.
similarity_threshold = 0.5

As you can see from the above code, the random seed is set to ensure that the random numbers generated are reproducible and its value is set to `30031`. The variable R is assigned a constant value of **1000001**, which is a large prime number for the hash function slot limitation while simultaneously ensures the hash collision encounterment. The similarity threshold is set to **0.5** and it refers to the exact Jaccard Similarity Index and not the min hash signature index.

The next step is the creation of the `hash_gen_init` function, which is used to generate min-hash signatures for users based on the movies they have seen. These min-hash signatures are later used for Locality Sensitive Hashing (LSH) to find similar users efficiently. The function takes three inputs: `data_frame`, which is the input dataframe, `num_hash_functions`, which represents the number of hash functions to generate, and `R`, which is a range limit used in the hash function calculations.

More specifically, we first extract the distinct `userids` from the dataframe and store them in `distinct_user_ids`. Then, the dictionary `user_movies` is initialized to store the movies seen by each user, by iterating over each user and retrieving their respective movies from the dataframe, and then assigning the movies to the user in the dictionary.

The below function also computes the min-hash signatures for each user. It iterates over the movies ids seen by each user, converts them to a set so as to have duplicates removed, and initializes the signature list with high values. For each movie, the function computes the hash values using the hash functions and updates the corresponding position in the signature list if the hash value is smaller - which will be for sure smaller from the initial ones that are out of bounce (R + 1). After computing the min-hash signature for a user, the signature list is appended to the `min_hash_signatures` list and finally the function returns the `min_hash_signatures` list containing the computed **min-hash signatures for all users**.

In [19]:
def hash_gen_init(data_frame, num_hash_functions, R):
    
    # Get distinct user ids from the imported dataframe.
    distinct_user_ids = set(data_frame['userid'])

    # Initialize a dictionary to store the movies seen by each user.
    user_movies = {}

    # Loop through users.
    for user_id in distinct_user_ids:
        
        # Get the respective movies for the current user.
        movies_seen = data_frame[data_frame['userid'] == user_id]['movieid']

        # Assign the movies to the current user.
        user_movies[user_id] = movies_seen.values

    # Generate random values 'a' and 'b' for hash functions.
    random_a = random.sample(range(1, R), num_hash_functions)
    random_b = random.sample(range(0, R), num_hash_functions)

    # Create a list of tuples representing the hash functions.
    hash_functions = [(a, b) for a, b in zip(random_a, random_b)]

    # Initialize a list to store the min-hash signatures for each user.
    min_hash_signatures = []

    # Compute min-hash signatures for each user.
    for movies_seen in user_movies.values():
        movies_seen = set(movies_seen)
        
        # Initialize signature with high values (could also be inf but R + 1 do the same).
        signature = [R + 1 for _ in range(num_hash_functions)]  

        # Compute the hash values and update the signature.
        for movie_id in movies_seen:
            for i, (a, b) in enumerate(hash_functions):
                hash_value = (a * movie_id + b) % R
                if hash_value < signature[i]:
                    signature[i] = hash_value

        min_hash_signatures.append(signature)

    return min_hash_signatures

The purpose of this function is to prepare the data for LSH by generating min-hash signatures. LSH allows finding similar users efficiently by grouping them based on the similarity of their min-hash signatures in different bands and then assessing their true similarity using their initial representations. The task asks to perform LSH using two different instances, each with a specific number of bands and rows per band. The goal is to find pairs of users with a similarity of at least 0.5 and report the number of true positive pairs and the number of similarity evaluations performed. This process is repeated for 5 different runs using different generated hash functions.

By using LSH and min-hash signatures, we gain efficiency in finding similar users by reducing the number of pairwise similarity evaluations. LSH allows us to narrow down the search space by focusing on users who have the same min-hash signatures in at least one band and as we learn through the lectures this significantly reduces the number of users' pairs that need to be compared for similarity evaluation. Thus, LSH provides an optimized probabilistic mechanism of finding similar users compared to directly comparing all possible users'pairs available in a dataset.

After this, the next step is to create the `jaccard_similarity`, as we earlier defined it also, but not as a separate function. We choose to do so now in order to enhance the code's clarity and take advantage of the object oriented programming capabilities.

To be more precise, the `jaccard_similarity` function computes the Jaccard coefficient between two sets. The function takes two input sets, set1 and set2, for which we want to calculate the Jaccard coefficient. It first finds the intersection of set1 and set2 using the intersection method, which returns a set containing the common elements between the two sets. It then finds the union of set1 and set2 using the union method, which returns a set containing all unique elements from both sets. The Jaccard coefficient is computed by dividing the length of the intersection set by the length of the union set. 

In other words, Jaccard similarity using the formula: 

`Jaccard Similarity = Intersection / Union`

Finaly, the computed Jaccard coefficient is returned as the output of the function and is a measure of similarity between two sets and is calculated as the size of the intersection divided by the size of the union. It ranges from **0 to 1, where 0 indicates no similarity and 1 indicates complete similarity**.

In [20]:
def jaccard_similarity(set1, set2):

    # Compute the intersection of set1 and set2.
    intersection = set1.intersection(set2)

    # Compute the union of set1 and set2.
    union = set1.union(set2)

    # Compute the Jaccard coefficient.
    jaccard_coefficient = len(intersection) / len(union)
    
    return jaccard_coefficient

Afterwards, we implement a function to create hash tables for each band in the LSH. 

The generated hash tables will be used to efficiently identify potential candidate pairs of similar users based on the similarity of their min-hash signatures within a specific band.

In [21]:
def create_hash_tables(signature_matrix):

    # Get the number of users and bands from the signature matrix.
    num_users = signature_matrix.shape[0]
    num_bands = signature_matrix.shape[1]

    # Initialize an empty list of hash tables, one for each band.
    hash_tables = [{} for _ in range(num_bands)]

    # A for loop statement to iterate over each user.
    for i in range(num_users):
        
        # A for loop statement to iterate over each band.
        for b in range(num_bands):
            
            # Get the min-hash signature for the current band.
            band = signature_matrix[i, b, :]

            # Convert the signature to a hashable value.
            hash_value = tuple(band)

            # Check if the hash value exists in the hash table.
            if hash_value not in hash_tables[b]:
                
                # If the hash value is not present, initialize an empty list.
                hash_tables[b][hash_value] = []

            # Append the current user to the corresponding hash table entry.
            hash_tables[b][hash_value].append(i)

    return hash_tables

Then, we create the `evaluate_similarity` to find similar users based on their min-hash signatures and compute the number of true positives and similarity evaluations. The function gets as input the followings

- user_movies: A dictionary mapping user IDs to the movies they have seen.
- signature_matrix: The signature matrix containing min-hash signatures.
- hash_tables: A list of hash tables, where each hash table corresponds to a band.
- similarity_threshold: The threshold for similarity to consider two users as similar.

And returns a dictionary of similar user pairs with their similarity values, the number of true positives, and the total number of similarity evaluations.

In [22]:
def evaluate_similarity(user_movies, signature_matrix, hash_tables, similarity_threshold):

    # Get the number of users and bands from the signature matrix.
    num_users = signature_matrix.shape[0]
    num_bands = signature_matrix.shape[1]

    # Initialize variables to store the results.
    similar_users = {}
    true_positives = 0
    similarity_evaluations = 0

    # Iterate over each pair of users.
    for i in range(num_users):
        for j in range(i + 1, num_users):
            
            # Iterate over each band.
            for b in range(num_bands):
                
                # Get the min-hash signature for the current user and band.
                band = signature_matrix[i, b, :]

                # Convert the signature to a hashable value.
                hash_value = tuple(band)

                # Check if the hash value exists in the hash table for the current band.
                if hash_value in hash_tables[b]:
                    
                    # Check if the second user is in the list of potential similar users
                    if j in hash_tables[b][hash_value]:
                        
                        # Compute the Jaccard similarity between the two users.
                        s1 = set(user_movies[i + 1])
                        s2 = set(user_movies[j + 1])
                        similarity = jaccard_similarity(s1, s2)
                        similarity_evaluations += 1

                        # Check if the similarity exceeds the threshold.
                        if similarity >= similarity_threshold:
                            
                            # Update the true positives count and store the similar user pair.
                            true_positives += 1
                            key = str(i + 1) + " - " + str(j + 1)
                            similar_users[key] = similarity

    # Sort the similar user pairs based on similarity in descending order.
    similar_users = dict(sorted(similar_users.items(), key=lambda x: x[1], reverse=True))

    # Return the results.
    return similar_users, true_positives, similarity_evaluations

Afterwards, we implement the function `user_similarity_lsh` in order to compute user similarity using Locality Sensitive Hashing (LSH) algorithm. This functions takes as required inputs the dictionary mapping user IDs to the movies they have seen, `user_movies`, the given number of bands for LSH `num_bands`, the also given number of rows per band for LSH, `num_rows_per_band`, the large prime number `R`, and the threshold for similarity to consider two users as similar - 0.5.

The following function will output a dictionary of similar user pairs with their similarity values, the number of true positives, and the total number of similarity evaluations.

In [23]:
def user_similarity_lsh(user_movies, num_bands, num_rows_per_band, R, similarity_threshold):
    
    # Calculate the number of hash functions based on the number of bands and rows per band.
    num_hash_functions = num_bands * num_rows_per_band

    # Generate min-hash signatures for users.
    user_signatures = hash_gen_init(ratings_df, num_hash_functions, R)

    # Reshape the user signatures into a signature matrix.
    signature_matrix = np.array(user_signatures).reshape(-1, num_bands, num_rows_per_band)

    # Create hash tables using the signature matrix.
    hash_tables = create_hash_tables(signature_matrix)

    # Evaluate user similarity using LSH.
    similar_users, true_positives, similarity_evaluations = evaluate_similarity(user_movies, signature_matrix, hash_tables, similarity_threshold)

    # Return the results.
    return similar_users, true_positives, similarity_evaluations

Finally, we are now ready to execute the LSH Instance 1 with the specified parameters, compute the similar user pairs, and prints the results.

In [24]:
# LSH Instance 1: b = 25, r = 8.
num_bands_1 = 25
num_rows_per_band_1 = 8

# Compute user similarity using LSH Instance 1.
similar_users_1, true_positives_1, similarity_evaluations_1 = user_similarity_lsh(user_movies, num_bands_1, num_rows_per_band_1, R, similarity_threshold)

# Print init mesage.
print("LSH Instance 1 - Similar Users:\n")

# Print similar user pairs and their similarity values for LSH Instance 1 - last run.
for key, value in similar_users_1.items():
    print("Users {}: {}".format(key, value))

LSH Instance 1 - Similar Users:

Users 408 - 898: 0.8387096774193549
Users 328 - 788: 0.6729559748427673
Users 489 - 587: 0.6299212598425197
Users 674 - 879: 0.5217391304347826
Users 554 - 764: 0.5170068027210885


And the average results for 5 runs for the LSH Instance 1.

In [25]:
# Print the average number of true positives for LSH Instance 1
print("LSH Instance 1 - True Positives:", true_positives_1/5)

# Print the total number of similarity evaluations for LSH Instance 1
print("LSH Instance 1 - Similarity Evaluations:", similarity_evaluations_1)

LSH Instance 1 - True Positives: 2.8
LSH Instance 1 - Similarity Evaluations: 45


We follow the same procedure for the LSH Instance 2 with it's own specified parameters, and prints the results.

In [26]:
# LSH Instance 2: b = 40, r = 5.
num_bands_2 = 40
num_rows_per_band_2 = 5

# Compute user similarity using LSH Instance 2.
similar_users_2, true_positives_2, similarity_evaluations_2 = user_similarity_lsh(user_movies, num_bands_2, num_rows_per_band_2, R, similarity_threshold)

# Print init message.
print("LSH Instance 2 - Similar Users:\n")

# Print similar user pairs and their similarity values for LSH Instance 2 - last run.
for key, value in similar_users_2.items():
    print("Users {}: {}".format(key, value))

LSH Instance 2 - Similar Users:

Users 408 - 898: 0.8387096774193549
Users 328 - 788: 0.6729559748427673
Users 489 - 587: 0.6299212598425197
Users 600 - 826: 0.5454545454545454
Users 451 - 489: 0.5333333333333333
Users 554 - 764: 0.5170068027210885
Users 197 - 826: 0.512987012987013
Users 197 - 600: 0.5
Users 800 - 879: 0.5


Last but not least, we print the average results for 5 runs for the LSH Instance 2.

In [27]:
# Print the average number of true positives for LSH Instance 2.
print("LSH Instance 2 - True Positives:", true_positives_2 / 5)

# Print the total number of similarity evaluations for LSH Instance 2.
print("LSH Instance 2 - Similarity Evaluations:", similarity_evaluations_2)

LSH Instance 2 - True Positives: 8.8
LSH Instance 2 - Similarity Evaluations: 1998


### Conclusions
---

LSH (Locality-Sensitive Hashing) is a technique used in data mining and similarity search to efficiently approximate similarity between high-dimensional data points. It is particularly useful for handling big datasets because it allows for scalable and efficient nearest neighbor search.

The main idea behind LSH is to hash data points in such a way that similar points have a high probability of being hashed into the same or nearby hash buckets. This enables the identification of potential similar pairs by focusing on a subset of the data instead of comparing all pairs, which is computationally expensive for large datasets. LSH is particularly effective for high-dimensional data because the curse of dimensionality makes traditional distance metrics less effective. In high-dimensional spaces, the density of data points becomes sparse, and the notion of distance breaks down. LSH helps mitigate this issue by focusing on the local structure of data points rather than global distances.

The main characteristics of hypertunning parameters of the LSH method is the number of bands and the number of rows in each band the signatures vectors will be divided to as this parameter determines the trade-off between the similarity accuracy attained and the complexity augmentation because of the increase in the possible simmilar pairs and consequently the similarity computations that are needed. By increasing 𝑏 in LSH increases recall by allowing more potential matches with lower similarity scores. By sIncreasing 𝑟 makes the match criteria stricter, leading to higher precision by restricting matches to higher similarity scores. These parameters can be adjusted to balance the trade-off between recall and precision based on the specific requirements of the application or similarity search task at hand.

In our solutions:
* *LHS Instance 1: b = 25, r = 8*

There are 45 similarity evaluations on average so the algorithm proposed **45** user pairs on average depending on 8 integers long tuple bands where a pair shares at least one band in common. From these evaluations about **3** pairs are identified correclty and are truly similar pairs over the 10 actual similar pairs identified form the initial set representations. The accuracy attained are rather low and this indicated that the bands and the presimilarity requirements are very strict and the pairs' selection is very stricted and limited.


* *LHS Instance 2: b = 40, r = 5*

As the bands' number increases and the number of rows in each band decreases , the recognizable pattern among bands becomes smaller and more often identified so the similarity evaluations augments and reaches **1998** for these trial pairs the true positives identified among them reaches the **8.8** pairs on average out of the 10 actual similar pairs with similarity above the threshold set. The computational complexity is increased as the possible pairs are increased but on the other hand the accuracy and the efficiency attained is increased.

**Therefore, by using LSH in enormous datasets, we gain computational speed at the cost of some accuracy and by decreasing the b parameter (or increasing the r parameter) we save computational force at the cost of the some accuracy in the actual similarity identifications.**

**Finally, we need to point out that the results of our analysis heavily depend on the set seed. During the development of the current notebook, we have found out for the first LSH instance values as low as 1.6 to 3.8 and for the second one from 7.6 to 9.4. However, we choose to present a mediocre output, in order to have a more fair analysis.**